In [15]:
# The first part of this notebook contains code that creates a torch dataset from the .csv saved from the dataframe_creation
# The code will ask for the .csv directory path
# The class AircraftImageDataset ultimately returns the image and the label for each index number.

# Then a DataLoader object is created that uses an AircraftImageDataset object from which to get the image and the label in 
# a form that is useable for models in torchvision


In [1]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms

from PIL import Image


In [13]:
dataframe_dir = input("Enter the base URL or directory path for the dataframe (csv) file: ").strip()

Enter the base URL or directory path for the dataframe (csv) file:  /Users/matthewboarts/Desktop/Capstone/fgvc-aircraft-2013b/data/labels


In [16]:
# This creates a dataframe from the previously saved dataframe (in the dataframe_creation notebook)
dataframe_path = os.path.join(dataframe_dir, 'comp_df_dataframe.csv')
ac_df = pd.read_csv(dataframe_path)



In [5]:
# The below code creates a particular dataframe for the train, test, val, and trainval sets. 
# The dataframes in this block contain all the columns from the original dataframe
# Additional dataframes will be easy to create for family, and manufacturer as desired.

# These lines filter all of the NaNs (notna) to retain only the rows desired.

variant_test = ac_df[pd.notna(ac_df['variant_test'])]
variant_train = ac_df[pd.notna(ac_df['variant_train'])]
variant_val = ac_df[pd.notna(ac_df['variant_val'])]
variant_trainval = ac_df[pd.notna(ac_df['variant_trainval'])]



In [6]:
#This block reduces the number of columns for each dataset

variant_test_df = variant_test[['image_index', 'variant_test', 'image_loc']]
variant_train_df = variant_train[['image_index', 'variant_train', 'image_loc']]
variant_val_df = variant_val[['image_index', 'variant_val', 'image_loc']]
variant_trainval_df = variant_trainval[['image_index', 'variant_trainval', 'image_loc']]

In [17]:
#This class creates a dataset from the dataframe in a form useable for torch

class AircraftImageDataset(Dataset):
    # the kwargs for __init__ are supplied in the dataloader when it calls this class
    # self == the object or instance created
    # dataframe == the dataframe containing the set of images/labels desired, i.e. train, test, val, trainval
    # transform == transform routines
    
    def __init__(self, dataframe, transform = None):
        self.data = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.data)

    # this returns the image and the label one row at a time
    def __getitem__(self, idx):
        columns = self.data.columns # list of columns
        column_name = self.data.columns[1] #from the list of columns, gets name of the set, i.e. train, test, val, trainval
        img_path = self.data.iloc[idx]['image_loc'] #from the list of columns, gets path for image (row by row -- increments with idx)
        label = self.data.iloc[idx][self.data.columns[1]] #from the list of columns, gets label (row by row -- increments with idx)
        image = Image.open(img_path).convert('RGB') #this is where the actual image file enters the code for the first time

        if self.transform: # applies the transform routines (if any) defined in __init__
            image = self.transform(image)

        return image, label
        

In [18]:
#these are the transform routines and will be an important subject of changing parameters
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])


In [19]:
# This calls the AircraftImageDataset, note the kwargs, change the set of data by changing the dataframe)
# Because the transform is applied before returning an image, and in this case the transform changes the image
# to a tensor, it will return a tensor and a label for each image
dataset = AircraftImageDataset(dataframe=variant_train_df, transform = transform)

In [20]:
# This provides the data to the torch model
# Once I have this in proper form you will be able to increase the num_workers as appropriate for your CPU
dataloader = DataLoader(dataset, batch_size = 32, shuffle=True, num_workers=0)

In [22]:
# Here is a loop to show a sample of the data. If the break was not there, it would return groups of 32 until the data
# is exhausted. It has groups of 32 because of the batch_size value in the dataloader kwargs.
i = 0
for img_data, label in dataloader:
    print("Image Data:", img_data)
    print("Label:", label)
    i = i +1
    if i == 3:
        break

Image Data: tensor([[[[0.6196, 0.6196, 0.6235,  ..., 0.5922, 0.5882, 0.5882],
          [0.6196, 0.6196, 0.6235,  ..., 0.5882, 0.5882, 0.5882],
          [0.6196, 0.6235, 0.6235,  ..., 0.5882, 0.5882, 0.5843],
          ...,
          [0.1882, 0.1216, 0.0667,  ..., 0.1569, 0.2039, 0.1569],
          [0.1843, 0.1373, 0.2039,  ..., 0.1647, 0.2902, 0.0706],
          [0.1412, 0.1137, 0.1373,  ..., 0.2078, 0.2039, 0.0039]],

         [[0.6392, 0.6392, 0.6392,  ..., 0.6196, 0.6157, 0.6157],
          [0.6353, 0.6392, 0.6392,  ..., 0.6157, 0.6157, 0.6157],
          [0.6392, 0.6392, 0.6431,  ..., 0.6157, 0.6157, 0.6157],
          ...,
          [0.2353, 0.1647, 0.1098,  ..., 0.2039, 0.2627, 0.2000],
          [0.1961, 0.1451, 0.2196,  ..., 0.1882, 0.3569, 0.0824],
          [0.1608, 0.1294, 0.1569,  ..., 0.2353, 0.2431, 0.0078]],

         [[0.6549, 0.6549, 0.6549,  ..., 0.6431, 0.6431, 0.6471],
          [0.6549, 0.6549, 0.6588,  ..., 0.6471, 0.6431, 0.6431],
          [0.6549, 0.6549, 0.6